In [5]:
import os
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras import optimizers
from tensorflow.keras import backend as K
from tensorflow.keras import utils
import statsmodels.tsa.stattools as sttl

## Environment

In [10]:
class Environment():

    def __init__(
        self,

        # date range
        start_date,
        end_date,

        # name of two stocks
        x_stock_name,
        y_stock_name,

        # estimation / episode info
        estimation_win=120,
        episode_win=30,

        # trade info
        original_pv=1000,
        transaction_cost=0.001425,
        stop_loss=None,
        take_profit=None
    ):
        # prepare training data
        self.x_train, self.y_train = self.prepare_data(x_stock_name, y_stock_name, start_date, end_date)
        self.ratio_train = self.x_train / self.y_train

        # estimation / episode info
        # episode = [begin, end)
        self.estimation_win = estimation_win
        self.episode_win = episode_win
        self.episode_begin = None
        self.episode_end = None
        self.episode_idx = None

        # trade info
        # pv = cash + (x_price * x_count) + (y_price * y_count)
        self.original_pv = original_pv
        self.transaction_cost = transaction_cost
        self.stop_loss = stop_loss
        self.take_profit = take_profit
        self.cash = self.original_pv
        self.x_count = 0
        self.y_count = 0
        self.agent_return = None
        self.agent_pv = None

        # baseline info (used to calculate baseline reward of a step)
        self.baseline_cash = self.original_pv
        self.baseline_x_count = 0
        self.baseline_y_count = 0
    


    # prepare training data
    def prepare_data(self, name1, name2, start_date, end_date):

        # intersect two stocks price on date
        x = pd.read_csv("data/{}.csv".format(name1))
        y = pd.read_csv("data/{}.csv".format(name2))
        x_y = pd.merge(left=x, right=y, how="inner", on="Date")

        i = x_y.index[x_y["Date"] == start_date][0]
        j = x_y.index[x_y["Date"] == end_date][0]
        x_y = x_y.iloc[i:j+1, :]

        # select date range
        x = np.array(x_y["Close_x"])
        y = np.array(x_y["Close_y"])
        print('spread mean: ', np.mean(x/y))

        
        # calculate cointegration p-value of training and testing data
        score, self.train_coint, _ = sttl.coint(x, y)

        return x, y

    
    # reset environment
    def reset(self):

        # info
        print("Number of day: ", len(self.x_train))
        print("P-value: ", self.train_coint)

        # trade info initialization
        self.cash = self.original_pv
        self.x_count = 0
        self.y_count = 0
        self.agent_return = 0
        self.agent_pv = 0

        # baseline info initialization (used to calculate baseline reward of a step)
        self.baseline_cash = self.original_pv
        self.baseline_x_count = 0
        self.baseline_y_count = 0

        # random sample begin of episode
        idx_list = list(range(self.estimation_win, len(self.x_train)-self.episode_win+1))
        self.episode_begin = random.sample(idx_list, 1)[0]
        self.episode_end = self.episode_begin + self.episode_win
        self.episode_idx = self.episode_begin


        # return initial state to agent
        sample = []
        x = np.array(self.x_train[self.episode_idx-self.estimation_win:self.episode_idx])
        y = np.array(self.y_train[self.episode_idx-self.estimation_win:self.episode_idx])
        
        x_y = np.concatenate((x, y))
        x_y_mu = np.mean(x_y)
        x_y_std = np.std(x_y)

        x = (x - x_y_mu) / x_y_std
        y = (y - x_y_mu) / x_y_std

        sample.append(x)
        sample.append(y)
        sample = np.array(sample)
        sample = np.transpose(sample)

        return sample
    
    
    # step next state
    def step(self, action):

        # calculate reward
        reward, current_pv = self.agent_reward(action)

        # check if state ends
        done = None
        if self.episode_idx == self.episode_end - 1 or current_pv < 0:
            done = True
        else:
            done = False
            self.episode_idx += 1
        
        # generate next state
        if done == False:
            sample = []
            x = np.array(self.x_train[self.episode_idx-self.estimation_win:self.episode_idx])
            y = np.array(self.y_train[self.episode_idx-self.estimation_win:self.episode_idx])
            x_y = np.concatenate((x, y))
            x_y_mu = np.mean(x_y)
            x_y_std = np.std(x_y)

            x = (x - x_y_mu) / x_y_std
            y = (y - x_y_mu) / x_y_std

            sample.append(x)
            sample.append(y)
            sample = np.array(sample)
            sample = np.transpose(sample)
        else:
            sample = np.zeros((self.estimation_win, 2))

        return sample, reward, done



    
    # calculate agent reward
    def agent_reward(self, action):

        '''
        action: 
        0: buy 1 x sell ratio y
        1: sell 1 x buy ratio y
        2: hold
        '''

        x_price = round(self.x_train[self.episode_idx-1], 6)
        y_price = round(self.y_train[self.episode_idx-1], 6)
        multiplier = round(self.ratio_train[self.episode_idx-1], 6)
        multiplier_constant = 1

        # buy 1 x sell ratio y
        if action == 0:

            # close perious position first
            if self.x_count < 0 and self.y_count > 0:
                self.cash += self.x_count * x_price
                self.cash -= abs(self.x_count * x_price) * self.transaction_cost
                self.x_count = 0

                self.cash += self.y_count * y_price
                self.cash -= self.y_count * y_price * self.transaction_cost
                self.y_count = 0
                

            # buy 1 x
            self.cash -= x_price * multiplier_constant
            self.cash -= x_price * multiplier_constant * self.transaction_cost 
            self.x_count += 1 * multiplier_constant

            # sell ratio y
            self.cash += multiplier * multiplier_constant * y_price
            self.cash -= (multiplier * multiplier_constant * y_price) * self.transaction_cost
            self.y_count -= multiplier * multiplier_constant

        # sell 1 x buy ratio y
        elif action == 1:

            # close perious position first
            if self.x_count > 0 and self.y_count < 0:
                self.cash += self.x_count * x_price
                self.cash -= self.x_count * x_price * self.transaction_cost
                self.x_count = 0

                self.cash += self.y_count * y_price
                self.cash -= abs(self.y_count * y_price) * self.transaction_cost
                self.y_count = 0

            # sell 1 x
            self.cash += x_price * multiplier_constant
            self.cash -= x_price * multiplier_constant * self.transaction_cost
            self.x_count -= 1 * multiplier_constant

            # buy ratio y
            self.cash -= multiplier * multiplier_constant * y_price
            self.cash -= (multiplier * multiplier_constant * y_price) * self.transaction_cost
            self.y_count += multiplier * multiplier_constant
        
        # calculate reward of this action
        current_pv = self.cash + (x_price * self.x_count) + (y_price * self.y_count)
        next_x_price = round(self.x_train[self.episode_idx], 6)
        next_y_price = round(self.y_train[self.episode_idx], 6)
        tomorrow_pv = self.cash + (next_x_price * self.x_count) + (next_y_price * self.y_count)
        action_reward = tomorrow_pv - current_pv
        # action_reward -= self.calc_baseline_reward(np.array(self.x_train[self.episode_idx-self.estimation_win:self.episode_idx]), np.array(self.y_train[self.episode_idx-self.estimation_win:self.episode_idx]))
        # action_reward -= self.calc_index_reward()
        

        
        # if exceed stop loss / take profit threshold, close position
        if (self.stop_loss != None and (current_pv-self.original_pv)/self.original_pv < self.stop_loss) or (self.take_profit != None and (current_pv-self.original_pv)/self.original_pv > self.take_profit):
        
            # close position for action 1
            if self.x_count > 0 and self.y_count < 0:
                self.cash += self.x_count * x_price
                self.cash -= self.x_count * x_price * self.transaction_cost
                self.x_count = 0

                self.cash += self.y_count * y_price
                self.cash -= abs(self.y_count * y_price) * self.transaction_cost
                self.y_count = 0

            # close position for action 2
            if self.x_count < 0 and self.y_count > 0:
                self.cash += self.x_count * x_price
                self.cash -= abs(self.x_count * x_price) * self.transaction_cost
                self.x_count = 0

                self.cash += self.y_count * y_price
                self.cash -= self.y_count * y_price * self.transaction_cost
                self.y_count = 0


        # check if last 2 day in episode
        if self.episode_idx == self.episode_end - 1 or current_pv < 0:
            self.agent_return = (tomorrow_pv - self.original_pv) / self.original_pv
            self.agent_pv = tomorrow_pv

        return action_reward, current_pv

## Double DQN Agent

In [11]:
class DoubleDQNAgent():

    def __init__(
        self,

        state_dim,
        action_dim,
        learning_rate,

        exploration_rate,
        exploration_decay,
        exploration_min,

        replay_buffer_size,

        batch_size,
        target_model_update_fqy,

        gamma
    ):

        self.state_dim = state_dim
        self.action_dim = action_dim
        self.learning_rate = learning_rate
        self.main_model = self.load_model("best_model_NFLX_TMUS_120.hdf5")
        # self.main_model = self.load_model2("ADBE & MSFT/normal/2013-2017/train/weight/main_model_1400_episode.hdf5")
        # self.main_model = models.load_model("GOOG & AMZN/base/train/weight/main_model_900_episode.hdf5")
        self.main_model.compile(loss="mse", optimizer=optimizers.Adam(learning_rate=self.learning_rate))
        self.main_model.summary()
        self.target_model = self.build_model()
        self.target_model.summary()
        self.update_target_model()

        self.batch_size = batch_size
        self.main_model_update_counter = 0
        self.target_model_update_fqy = target_model_update_fqy

        self.exploration_rate = exploration_rate
        self.exploration_decay = exploration_decay
        self.exploration_min = exploration_min

        self.replay_buffer = list(range(replay_buffer_size))
        self.replay_idx = 0
        self.replay_buffer_size = replay_buffer_size

        self.gamma = gamma

        


    def build_model(self):

        model = None

        inputs = layers.Input(shape=(self.state_dim, 2))
        x = layers.Conv1D(filters=256, kernel_size=14, activation="relu")(inputs)
        x = layers.MaxPooling1D(pool_size=2)(x)
        x = layers.Conv1D(filters=256, kernel_size=10, activation="relu")(x)
        x = layers.MaxPooling1D(pool_size=2)(x)
        x = layers.Conv1D(filters=256, kernel_size=7, activation="relu")(x)
        x = layers.MaxPooling1D(pool_size=2)(x)
        x = layers.Conv1D(filters=128, kernel_size=5, activation="relu")(x)
        x = layers.MaxPooling1D(pool_size=2)(x)
        x = layers.Flatten()(x)
        
        x = layers.Dense(128, activation="relu")(x)
        x = layers.Dense(64, activation="relu")(x)
        x = layers.Dense(32, activation="relu")(x)
        
        values = layers.Dense(3, activation='linear')(x)
        a = layers.Dense(3, activation='linear')(x)
        mean = layers.Lambda(lambda i: K.mean(i, axis=1, keepdims=True))(a)
        advantage = layers.Subtract()([a, mean])
        q = layers.Add()([values, advantage])
        
        model = models.Model(inputs, q)

        return model

    def load_model(self, path):

        base_model = models.load_model(path)
        model = models.Sequential()
        for layer in base_model.layers[1:10]:
            model.add(layer)
        
        for layer in model.layers:
            layer.trainable = False
            
        inputs = layers.Input(shape=(self.state_dim, 2))
        x = model(inputs)
        x = layers.Dense(128, activation="relu")(x)
        x = layers.Dense(64, activation="relu")(x)
        x = layers.Dense(32, activation="relu")(x)
        
        values = layers.Dense(3, activation='linear')(x)
        a = layers.Dense(3, activation='linear')(x)
        mean = layers.Lambda(lambda i: K.mean(i, axis=1, keepdims=True))(a)
        advantage = layers.Subtract()([a, mean])
        q = layers.Add()([values, advantage])
        
        model = models.Model(inputs=inputs, outputs=q)

        return model

    '''
    def load_model2(self, path):

        model = models.load_model(path)
        new_model = models.Sequential([
            layers.Input(shape=(self.state_dim, 2))
        ])

        for layer in model.layers[:9]:
            layer.trainable = False
            new_model.add(layer)

        new_model.add(layers.Dense(256, activation="relu"))
        new_model.add(layers.Dense(128, activation="relu"))
        new_model.add(layers.Dense(16, activation="relu"))
        new_model.add(layers.Dense(self.action_dim, activation="linear"))
        
        new_model.summary()
        return new_model
    '''


    def update_target_model(self):

        self.target_model.set_weights(self.main_model.get_weights())

    
    def sample_action(self, state):

        if np.random.rand() < self.exploration_rate:
            action = np.random.choice(self.action_dim)
        else:
            state = np.reshape(state, (1, self.state_dim, 2))
            q_values = self.main_model.predict(state)
            action = np.argmax(q_values[0])

        return action

    
    def remember(self, state, action, reward, new_state, done):

        exp = (state, action, reward, new_state, done)
        idx = self.replay_idx % self.replay_buffer_size
        self.replay_buffer[idx] = exp
        self.replay_idx += 1


    
    def update_main_model(self):

        if self.replay_idx < self.batch_size:
            return

        candidate_idx = np.random.choice(range(min(self.replay_idx, self.replay_buffer_size)), [self.batch_size])

        batch_state = []
        batch_qvalue = []

        for idx in candidate_idx:

            current_exp = self.replay_buffer[idx]
            state, action, reward, new_state, done =current_exp

            q_values = self.main_model.predict(np.reshape(state, (1, self.state_dim, 2)))[0]
            target_q = self.get_target_q(new_state, reward)

            if done == True:
                target_q = reward
            
            q_values[action] = target_q

            batch_state.append(state)
            batch_qvalue.append(q_values)

        self.main_model.fit(
            x=np.array(batch_state),
            y=np.array(batch_qvalue),
            epochs=1,
            batch_size=self.batch_size,
            verbose=0
        )

        # decrease exploration, increase exploitation
        self.exploration_rate = max(self.exploration_min, self.exploration_rate*self.exploration_decay)

        # update target model
        self.main_model_update_counter += 1
        if self.main_model_update_counter % self.target_model_update_fqy == 0:
            self.update_target_model()


    
    def get_target_q(self, new_state, reward):

        action = self.main_model.predict(np.reshape(new_state, (1, self.state_dim, 2)))[0]
        action = np.argmax(action)

        target_q = self.target_model.predict(np.reshape(new_state, (1, self.state_dim, 2)))[0][action]

        target_q *= self.gamma
        target_q += reward

        return target_q

## Main

In [12]:
TOTAL_EPISODE = 2001
CURRENT_EPISODE = 0
STATE_DIM = 120

In [14]:
env = Environment(
    # date range
    start_date="2011-01-03",
    end_date="2015-12-30",

    # name of two stocks
    x_stock_name="NFLX",
    y_stock_name="TMUS",

    # estimation / episode info
    estimation_win=STATE_DIM,
    episode_win=30,

    # trade info
    original_pv=1000,
    transaction_cost=0.001425,
    stop_loss=None,
    take_profit=None,
)

agent = DoubleDQNAgent(
    state_dim=STATE_DIM,
    action_dim=3,
    learning_rate=0.0003,

    exploration_rate=0.99,
    exploration_decay=0.998,
    exploration_min=0.05,

    replay_buffer_size=30*600,

    batch_size=64,
    target_model_update_fqy=10,

    gamma=0.9
)

spread mean:  0.7608787142411229
Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 120, 2)]     0                                            
__________________________________________________________________________________________________
sequential_1 (Sequential)       (None, 256)          1286016     input_3[0][0]                    
__________________________________________________________________________________________________
dense_10 (Dense)                (None, 128)          32896       sequential_1[0][0]               
__________________________________________________________________________________________________
dense_11 (Dense)                (None, 64)           8256        dense_10[0][0]                   
___________________________________________________________

In [ ]:
episode_reward_history = []
episode_return_history = []
episode_pv_history = []

In [ ]:
while CURRENT_EPISODE < TOTAL_EPISODE:

    episode_reward = 0
    is_done = False
    current_state = env.reset()

    print("Current Episode: {}".format(str(CURRENT_EPISODE)))

    while is_done == False:

        action = agent.sample_action(current_state)
        new_state, reward, is_done = env.step(action)
        agent.remember(current_state, action, reward, new_state, is_done)

        episode_reward += reward
        current_state = new_state
    
    print("#{} Episode Reward: {}".format(str(CURRENT_EPISODE), str(episode_reward)))
    episode_reward_history.append(episode_reward)
    episode_return_history.append(env.agent_return)
    episode_pv_history.append(env.agent_pv)
    print()

    agent.update_main_model()
    
    if CURRENT_EPISODE % 100 == 0:
        models.save_model(agent.main_model, "main_model_{}_episode.hdf5".format(str(CURRENT_EPISODE)))
        
    CURRENT_EPISODE += 1

In [ ]:
plt.figure(figsize=(10, 6))
plt.title("Episode PV History")
plt.xlabel("Episode")
plt.ylabel("PV")
plt.plot(episode_pv_history, label="Raw")
plt.axhline(y=1000, color="black")
plt.legend()

In [ ]:
plt.figure(figsize=(10, 6))
plt.title("Episode Reward History")
plt.xlabel("Episode")
plt.ylabel("Reward")
plt.plot(episode_reward_history, label="Raw")
plt.axhline(y=0, color="black")
plt.legend()

In [ ]:
plt.figure(figsize=(10, 6))
plt.title("Episode PV History")
plt.xlabel("Episode")
plt.ylabel("PV")

episode_pv_history_ma = []
for i in range(50, len(episode_pv_history)):
    episode_pv_history_ma.append(np.mean(episode_pv_history[i-50:i]))

plt.plot(episode_pv_history_ma, label="50 MA")
plt.legend()

In [ ]:
plt.figure(figsize=(10, 6))
plt.title("Episode Reward History")
plt.xlabel("Episode")
plt.ylabel("Reward")

episode_reward_history_ma = []
for i in range(100, len(episode_reward_history)):
    episode_reward_history_ma.append(np.mean(episode_reward_history[i-100:i]))

plt.plot(episode_reward_history_ma, label="10 MA")
plt.axvline(x=0, color="black")

In [ ]:
plt.figure(figsize=(10, 6))
plt.title("Episode PV History")
plt.xlabel("Episode")
plt.ylabel("PV")

plt.plot(episode_pv_history, label="Raw")
plt.plot(episode_pv_history_ma, label="10 MA")

plt.axhline(y=1000, color="black")
plt.legend()